In [ ]:
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
DATA_DIR = "../data/"

In [6]:
def load_proton_a_images(n=None):
    data_a = pd.read_pickle(os.path.join(DATA_DIR, "proton_images2a.pkl"))
    data_b = pd.read_pickle(os.path.join(DATA_DIR, "proton_images2b.pkl"))
    data_c = pd.read_pickle(os.path.join(DATA_DIR, "proton_images2c.pkl"))
    return np.r_[data_a, data_b, data_c][:n]

def load_particle_data_a(n=None):
    data_a = pd.read_pickle(os.path.join(DATA_DIR, "particles_cond_with_mass2a.pkl"))
    data_b = pd.read_pickle(os.path.join(DATA_DIR, "particles_cond_with_mass2b.pkl"))
    data_c = pd.read_pickle(os.path.join(DATA_DIR, "particles_cond_with_mass2c.pkl"))
    return pd.concat([data_a, data_b, data_c], axis=0)[:n]

In [32]:
N_SAMPLES_T0_LOAD = None

data_p = load_proton_a_images(N_SAMPLES_T0_LOAD)
print('Loaded Proton: ',  data_p.shape)

data_cond = load_particle_data_a(N_SAMPLES_T0_LOAD)
print('Loaded cond: ',  data_cond.shape)

Loaded cond:  (511810, 13)


## Calculate Proton Photon Sum

In [30]:
data_cond['proton_photon_sum'] = np.sum(data_p, axis=(1,2))

ValueError: No axis named (1, 2) for object type DataFrame

In [33]:
data_cond.head()

,Unnamed: 0,Pdg,Energy,Vx,Vy,Vz,Px,Py,Pz,neutron_photon_sum,mass,charge,proton_photon_sum
0,0,22,513.318,1.454300e-08,3.650510e-08,-0.002731,0.035456,-0.051821,-513.318,291.0,0.000000,0.0,0.0
1,1,22,1304.340,-1.831820e-05,1.555780e-05,0.050293,-0.509192,0.461106,1304.340,8.0,0.000000,0.0,9.0
2,2,22,1014.720,-1.831820e-05,1.555780e-05,0.050293,-0.335473,0.256277,1014.720,130.0,0.000000,0.0,0.0
3,3,-211,325.158,0.000000e+00,0.000000e+00,0.000000,0.269915,-0.081921,325.157,0.0,139.570610,-1.0,5.0
4,4,2112,3192.380,0.000000e+00,0.000000e+00,0.000000,0.022422,-0.182957,-3192.380,1256.0,939.565413,0.0,16.0


## Filter Photon Sum

In [34]:
## Filter Photon Sum
def filter_photon_sum(data_p, data_conditional, min_photon_sum=10, max_photon_sum=None):
    """
    Returns particle data and responses of experiments with minimum sum of photons.
    """
    if max_photon_sum:
        mask = (data_conditional.proton_photon_sum >= min_photon_sum)\
               & (data_conditional.proton_photon_sum <= max_photon_sum)
    else:
        mask = (data_conditional.proton_photon_sum >= min_photon_sum)
    return data_p[mask], data_conditional[mask].reset_index(drop=True)


MIN_PHOTON_SUM = 15
MAX_PHOTON_SUM = None
data_p = pd.DataFrame()
data_p, data_cond = filter_photon_sum(data_p, data_cond, min_photon_sum=MIN_PHOTON_SUM, max_photon_sum=MAX_PHOTON_SUM)
print(data_p.shape, data_cond.shape)
print("Min proton photon sum: ", data_cond.proton_photon_sum.min())

C:\Users\PB\AppData\Local\Temp\ipykernel_19084\1219911072.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return data_p[mask], data_n[mask], data_conditional[mask].reset_index(drop=True)


(0, 0) (48714, 44, 44) (48714, 13)
Min proton photon sum:  15.0 Min proton photon sum:  15.0


## Save filtered data

In [20]:
# calculate min max proton, neutron sum
photon_sum_proton_min, photon_sum_proton_max = int(data_cond.proton_photon_sum.min()), int(data_cond.proton_photon_sum.max())

In [21]:
photonsums_intervals_str = f"{photon_sum_proton_min}_{photon_sum_proton_max}"

'15_2133_15_3273'

In [18]:
# save conditional data
data_cond.to_pickle(f"../data/data_cond_photonsum_{photonsums_intervals_str}.pkl")

In [22]:
# save proton data
with open(f"../data/data_proton_photonsum_{photonsums_intervals_str}.pkl", 'wb') as f:
    pickle.dump(data_p, f)